In [1]:
from __future__ import print_function, division
from scipy.optimize import fsolve #to get the initial value of vecp0
import numpy as np
from scikits.odes import dae
import pandas as pd
import time
import cantera as ct
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
from scipy.optimize import fsolve, root
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['font.family'] = 'Times New Roman'
colors = brewer2mpl.get_map('Set2','qualitative',8).mpl_colors

The absorption  one-step reaction H2O (g) + O (s) <=> H2O (s) is used as the simple mechanism for simulating the plug flow reactor with surface chemistry. Where g and s mean the gas phase and surface phase respectively and O stands for the empty site.
## 1. IDA solver to calculate Zk

In [2]:
%%latex
Since the heterogeneous production rates for gas species $\dot{g_k}$ need to be determined based on the composition of surface and gas , we first need to calculate the site fraction of surface species $Z_k$ of $K_s$ species
\begin{align}
    \frac{dZ_k}{dt} &= \frac{\dot{s_k}\sigma_k}{\Gamma}\\
    \dot{s_k} &= 0 ~~~\text{at steady state for 1 to $K_s$ - 1 species}\\                 
    \sum_{phase}{Z_k}&= 1 
\end{align}
The procedure is described as follows:<br>
1. IDA solver is applied to solve the DAE which includes the following first and third equations. In order to minimize the error, the species with initial large site fraction should be determined by the algebraic equation, site fractions of other species can be obtained by the differential equation.<br>

2. Use the while loop to find the site fractions at steady state. When $\dot{s_k}$ = 0 for $K_s$-1 species, the system reraches steady state

<IPython.core.display.Latex object>

### 1.1 check the Zk at the entrance of the tube

In [8]:
R =  8.314 #J / mol. K
D = 5.08 * 10**-2 #diameter of the tube [m]
Ac = np.pi * D**2/4 # cross section of the tube [m]
Vdot = 588 * 10**-6/60 # volumetric rate [m^3/s] 
u0 = Vdot * Ac
nu = 1.418E-7 #m2/s
Gamma = 0.417E-7 #mol/m^2 site desity
sigma_k = [0, 2] #site occupancy number
perim = np.pi * D #perimeter of the tube

p = ct.one_atm 
tinlet = 500.0
tsurf = 500.0
transport = 'Mix'
# composition of the inlet premixed gas
comp1 = 'H2O:1'
 
# create gas object
mech = '/Users/yuanjie/Dropbox/Cantera_intern/code/H2O_absorp/small_mec.cti'
gas = ct.Solution(mech,'gas')
gas.TPY = tinlet, p, comp1
gas.set_unnormalized_mass_fractions(gas.Y)

#create the interface object
surf_phase = ct.Interface(mech, 'oxide_surface',[gas])
surf_phase.TP = tsurf,p
surf_phase.set_unnormalized_coverages
sigma_k = [0,2]

Z0 = np.array([1,0])
Zp0 = surf_phase.net_production_rates[-surf_phase.n_species:]*sigma_k[:surf_phase.n_species]/Gamma

def residual_dZkdt(t, Z, dotZ, result):
    """Z is a vector stored the site fraction of all surface species"""
    surf_phase.coverages = Z
    for i in range(surf_phase.n_species):
        result[i] = dotZ[i] - surf_phase.net_production_rates[-surf_phase.n_species+i]*sigma_k[i]/Gamma
    np.delete(result, np.argmax(Z0)) # remove the largest site fraction element
    result[np.argmax(Z0)] = np.sum(Z) - 1

"""
def residual_dZkdt(t, Z, dotZ, result):
    #Z is a vector stored the site fraction of all surface species
    surf_phase.coverages = Z
    for i in range(surf_phase.n_species-1):
        result[i] = dotZ[i] - surf_phase.net_production_rates[-surf_phase.n_species+i]*sigma_k[i]/Gamma
    result[surf_phase.n_species-1] = np.sum(Z) - 1
"""
solver = dae('ida', residual_dZkdt, 
             compute_initcond='yp0', #If yp0, then the differential variables (y of the ode system at time 0) will be used to solve for the derivatives of the differential variables, so yp0 will be calculated
             first_step_size=1e-18,
             atol=1e-6, #absolute tolerance for solution
             rtol=1e-6, #relative tolerance for solution
             algebraic_vars_idx=[np.argmax(Z0)], #If the given problem is of type DAE, some items of the residual
                    #vector returned by the 'resfn' have to be treated as
                    #algebraic equations, and algebraic variables must be defined.
                    #These algebraic variables are denoted by the position (index)
                    #in the state vector y.
                    #All these indexes have to be specified in the
                    #'algebraic_vars_idx' array.
             #compute_initcond_t0 = 60,#When calculating the initial condition, specifies the time
                                      # until which the solver tries to
                                      #get the consistent values for either y0 or yp0 relative to
                                      #the starting time. Positive if t1 > t0, negative if t1 < t0
             max_steps=5000,
             old_api=False)#Forces use of old api (tuple of 7) if True or
                    #new api (namedtuple) if False.
                    #Other options may require new api, hence using this should
                    #be avoided if possible.
t=0
"""
Partial_Surf_NetProdRate = surf_phase.net_production_rates[-surf_phase.n_species:]
np.delete(Partial_Surf_NetProdRate,np.argmax(Z0))
while np.any(Partial_Surf_NetProdRate!=np.zeros(surf_phase.n_species-1)):
"""
while np.any(surf_phase.net_production_rates[-surf_phase.n_species:]!=np.zeros(surf_phase.n_species)):
    t = t + 10
    solution = solver.solve(np.array([0,t]), Z0, Zp0)
    surf_phase.coverages = solution.values.y[1,:]
#check results with advance_to_coverage function, the results are the same
print(surf_phase.coverages)
print(surf_phase.net_production_rates)

[ 0.99701923  0.00298077]
[ 0.  0.  0.]


## 2. Calculate the thermal and kinetic properties along with the flow direction
### 2.1 Determine the initial value of those variables

In [4]:
%%latex
Since IDA solver needs input the initial value of variables and their primes. The following nonlinear equation system has been solved by the nonlinear solver.
\begin{align}
    u_0\rho_0' + \rho_0 u_0' - \frac{p'}{A_c}\sum^{K_g}\dot{s_k}W_k &= 0\\
    \rho_0 u_0 A_c Y_{k,0}' + Y_{k,0} p'\sum^{K_g}\dot{s_k}W_k - \dot{\omega_k}W_kA_c - \dot{s_k}W_k p' &=0 \\                 
    2\rho_0 u_0 u_0' + u_0^2\rho_0' + P_0' + \frac{8\rho_0 \nu \pi}{A_c} &=0\\
    -RT\rho_0' + \bar{W_0}P_0' - P_0\frac{\sum^{K_g}Y_k'/W_k}{(\sum^{K_g}Y_k/W_k)^2} &= 0 
\end{align}

<IPython.core.display.Latex object>

In [14]:
rho0 = gas.density
p0 = ct.one_atm
T = 500
def equations(vecp0):
    eq = np.empty(3+gas.n_species)
    eq[0] = vecp0[0]*rho0 + vecp0[1]*u0 - perim * np.sum(surf_phase.net_production_rates[:gas.n_species] * gas.molecular_weights)/Ac
    for i in range(gas.n_species):   
        eq[1+i] = rho0 * u0 * Ac * vecp0[2+i]\
                  + gas.Y[i] * perim * np.sum(surf_phase.net_production_rates[:gas.n_species] * gas.molecular_weights)\
                  - gas.net_production_rates[i] * gas.molecular_weights[i] * Ac\
                  - surf_phase.net_production_rates[i] * gas.molecular_weights[i] * perim
    eq[1+gas.n_species] = 2*rho0 * u0 * vecp0[0] + u0**2*vecp0[1] + vecp0[2+gas.n_species] + 8*rho0*u0*nu*np.pi/Ac
    eq[2+gas.n_species] = -R*T*vecp0[1] + gas.mean_molecular_weight*vecp0[2+gas.n_species] - p0*1/np.power(np.sum(gas.Y/gas.molecular_weights),2)*np.sum(vecp0[2:2+gas.n_species]/gas.molecular_weights)
    return eq
vec0 = np.append(np.append([u0, rho0], gas.Y),p0)
vecp_guess = np.array([0,0,0,0])
vecp0 = fsolve(equations, vecp_guess)  
vecp0

array([  3.00639846e-21,  -6.64594319e-14,  -3.42113883e-49,
        -1.53354185e-11])

### 2.2 IDA solver

In [39]:
#to calcualte the gas species
def residual(z, vec, vecp, result):
    """ vec = [u, rho, Y_k, p]"""
    T = 500
    gas.set_unnormalized_mass_fractions(vec[2:2+gas.n_species])
    gas.TPY = T,vec[2+gas.n_species],vec[2:2+gas.n_species]
    gas.density = vec[1]
    surf_phase.TP = T,vec[2+gas.n_species]
    surf_phase.set_unnormalized_coverages
    sigma_k = [0,2]
    Z0 = np.array([1,0])
    Zp0 = surf_phase.net_production_rates[-surf_phase.n_species:]*sigma_k[:surf_phase.n_species]/Gamma
    def residual_dZkdt(t, Z, dotZ, result):
        """Z is a vector stored the site fraction of all surface species"""
        surf_phase.coverages = Z
        for i in range(surf_phase.n_species):
            result[i] = dotZ[i] - surf_phase.net_production_rates[-surf_phase.n_species+i]*sigma_k[i]/Gamma
        np.delete(result, np.argmax(Z0)) # remove the largest site fraction element
        result[np.argmax(Z0)] = np.sum(Z) - 1

    solver = dae('ida', residual_dZkdt, 
             compute_initcond='yp0', 
             atol=1e-6, 
             rtol=1e-6,
             algebraic_vars_idx=[np.argmax(Z0)], 
             max_steps=5000,
             old_api=False)
    t=0
    while np.any(surf_phase.net_production_rates[-surf_phase.n_species:]!=np.zeros(surf_phase.n_species)):
        t = t + 10
        solution = solver.solve(np.array([0,t]), Z0, Zp0)
        surf_phase.coverages = solution.values.y[1,:]

    sdot_gas = surf_phase.net_production_rates[:gas.n_species]
    result[0] = vec[0]*vecp[1]+vec[1]*vecp[0]-perim*np.sum(sdot_gas*gas.molecular_weights)/Ac
    for k in range(gas.n_species):
        result[1+k] = vec[1]*vec[0]*Ac*vecp[2+k] + vec[2+k]*perim*np.sum(sdot_gas*gas.molecular_weights)\
                      - gas.net_production_rates[k]*gas.molecular_weights[k]*Ac\
                      - surf_phase.net_production_rates[k]*gas.molecular_weights[k]*perim
    result[1+gas.n_species] = 2*vec[1]*vec[0]*vecp[0]+vec[0]**2*vecp[1]+vecp[2+gas.n_species]+8*vec[1]*vec[0]*nu*np.pi/Ac
    result[2+gas.n_species] = vec[2+gas.n_species]*gas.mean_molecular_weight-vec[1]*R*T

In [42]:
solver = dae('ida', residual, 
             compute_initcond='yp0',
             first_step_size=1e-18,
             atol=1e-6,
             rtol=1e-6, 
             algebraic_vars_idx=[2+gas.n_species], 
             max_steps=5000,
             old_api=False)
solution = solver.solve(np.arange(0,0.2,0.1), vec0, vecp0)
solution

Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored
Exception TypeError: 'len() of unsized object' in 'scikits.odes.sundials.ida._res' ignored

SolverReturn(flag=-14, values=SolverVariables(t=None, y=None, ydot=None), errors=SolverVariables(t=0.0, y=array([  1.98629332e-08,   4.39089919e-01,   1.00000000e+00,
         1.01325000e+05]), ydot=array([  3.00639846e-21,  -6.64594319e-14,  -3.42113883e-49,
        -1.53354185e-11])), roots=SolverVariables(t=None, y=None, ydot=None), tstop=SolverVariables(t=None, y=None, ydot=None), message='The residual routine or the linear setup or solve routine had a recoverable error, but IDACalcIC was unable to recover.')